In [34]:
from __future__ import division
import numpy as np
import sys,json,itertools,re,argparse
from collections import Counter,defaultdict

In [35]:
from get_feats import * 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
#real example 
parse_samp = [["root", 0, 1], ["punct", 4, 2], ["nsubj", 4, 3], ["dep", 1, 4], ["punct", 4, 5], ["punct", 4, 6], ["root", 6, 7], ["compound", 9, 8], ["acl:relcl", 7, 9], ["dobj", 9, 10], ["punct", 7, 11], ["root", 11, 12]]
argspan = [u'6', u'7']
print len(parse_samp)

12


In [37]:
child2parent = {edge[2]:edge[1] for edge in parse_samp}
#is it true that each child can only have one parent?? 
print child2parent
print len(child2parent)

{1: 0, 2: 4, 3: 4, 4: 1, 5: 4, 6: 4, 7: 6, 8: 9, 9: 7, 10: 9, 11: 7, 12: 11}
12


In [20]:
#toy example 
argspan =[u'0', u'2']
parse_samp = [['deplabel', 4, 2], ['deplabel', 2, 3], ['deplabel', 3, 1]]
print len(parse_samp)

3


In [21]:
child2parent = {edge[2]:edge[1] for edge in parse_samp}
#is it true that each child can only have one parent?? 
print child2parent
print len(child2parent)

{1: 3, 2: 4, 3: 2}
3


In [40]:
def get_head(parse_samp, argspan):
    '''
    parse_samp::
    a single parse edges example e.g.
    parse_samp = [['deplabel', 4, 2], ['deplabel', 2, 3], ['deplable', 3, 1]]
    
    argspan::
    the starting and ending tokens for a single argument 
    argspan =[u'0', u'2']
    
    this returns the "head index" from the argspan
    by using the dependency edges given by parse_samp 
    '''
    votes = defaultdict(int)
    child2parent = {edge[2]-1:edge[1]-1 for edge in parse_samp} #need the -1 for parse to argspan aligning
    assert len(child2parent) == len(parse_samp), 'problem with parse tree structure??'
    
    as_start = int(argspan[0]) #starting index of argument text span
    as_end = int(argspan[1]) #ending index of argument text span 
    arg_tokrange =[x for x in xrange(as_start, as_end+1)]
     
    #make votes by iterating thru all the toks 
    for tok in arg_tokrange:
        #basecase tok is the head 
        child = tok 
        parent = child2parent[child]
        #go up the paths until you EXIT the token span of the current argument 
        while parent in arg_tokrange:
            child = parent
            parent = child2parent[child]
        head = child
        votes[head] += 1
    
    #then find the head index
    max_votes = max(votes.values())
    possible_heads = [k for k in votes.keys() if votes[k] == max_votes]
    head_idx = min(possible_heads) #IMPORTANT: ties go to the smallest index 
    
    return head_idx

get_head(parse_samp, argspan)

6